In [1]:
import pandas as pd

In [2]:
system_info = pd.read_csv('../data/interim/all_library_info.csv')
ecor_predictions = pd.read_csv('../data/interim/ecor_predictions_defensive_cat.csv')

/state/partition1/slurm_tmp/27646594.0.0/ipykernel_1627253/3275897380.py:2: DtypeWarning: Columns (21,27,35) have mixed types. Specify dtype option on import or set low_memory=False.
  ecor_predictions = pd.read_csv('../data/interim/ecor_predictions_defensive_cat.csv')


In [3]:
ecor_seq_assemblies = pd.read_csv('../data/interim/ecor_seq_assemblies.csv', names=['product_accession', 'seq_id', 'seq', 'assembly'])
ecor_seqs = ecor_seq_assemblies[['product_accession', 'seq']].drop_duplicates()

In [4]:
defensive_systems = system_info[system_info['defensive']]
len(defensive_systems)

45

In [10]:
exploded_systems = defensive_systems.copy()
exploded_systems['operon'] = exploded_systems['operon'].str.split(';')
exploded_systems = exploded_systems.explode('operon')
exploded_systems = exploded_systems[['operon', 'working_id', 'screened LB', 'fp', 'pBAD', 'library', 'random', 'curated', 'defensive', 'max_log_odds', 'category']]

In [11]:
ecor_predictions.merge(exploded_systems, how='inner', on='operon').drop_duplicates().shape

(73, 49)

In [12]:
merged_predictions = ecor_predictions.merge(exploded_systems, how='inner', on='operon').drop_duplicates().reset_index(drop=True)
merged_predictions['fake_start'] = (pd.Series([-1 if (strand == '-') else 1 for strand in merged_predictions['strand']])) * merged_predictions['start']
merged_predictions['protein_num'] = (merged_predictions.groupby('working_id')
                                     ['fake_start']
                                     .rank()
                                     .astype(int))
merged_predictions = (merged_predictions.merge(ecor_seqs, how='inner', on='product_accession'))

In [13]:
breif_predictions = merged_predictions[['protein_context_id', 'assembly', 'genomic_accession', 'start', 'end', 'strand',
                                        'product_accession', 'name', 'seq', 'product_length',
                                        'beaker_prediction', 'log_odds',
                                        'defense_homolog_names', 'sys_id', 
                                        'protein_num', 'hit_name', 'q_cov', 'Prob', 
                                        'operon', 'working_id', 'pBAD', 'library', 'defensive']]

In [14]:
with open('../data/interim/hit_proetins.faa', 'w') as f:
    for _, row in breif_predictions.iterrows():
        print('>' + row['product_accession'], file=f)
        print(row['seq'], file=f)

In [15]:
breif_predictions.to_csv('../data/interim/hit_seq_info.csv', index=False)